In [2]:
import os
import pandas as pd
import pyspark
from pyspark import SparkConf
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession 
import pyspark.sql.functions as f

# Step 1: Define Spark Configuration
conf = SparkConf() \
    .setAppName("PodProcessing") \
    .setMaster("local[35]") \
    .set("spark.executor.memory", "350g") \
    .set("spark.driver.memory", "350g")

# Step 2: Initialize SparkContext with the Configuration
sc = SparkContext(conf=conf)

# Step 3: Initialize SparkSession
spark = SparkSession(sc)

In [ ]:
!pip3.11 install pymarc poetry marctable

In [ ]:
!pip3.11 install --upgrade pip

In [ ]:
import os

# Add the directory to PATH
os.environ['PATH'] += os.pathsep + '/home/jovyan/.local/bin'

# Initial load only
Converts MARC to Parquet format for faster processing.

In [ ]:
import os
import glob
import tempfile
import logging
from pymarc import Record
from pymarc.marcxml import parse_xml_to_array

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def get_files():
    # Use recursive search to pick up .xml files anywhere under the given directory.
    # Filter out files that are not MARC XML (e.g. those containing 'opensearch' in their name)
    all_files = glob.glob('/home/jovyan/work/*.xml', recursive=True)
    files = [f for f in all_files if 'opensearch' not in os.path.basename(f).lower()]
    logger.info(f"Found {len(files)} marc files: {files}")
    return files

def process_file(file):
    logger.info(f"Processing file {file}")

    # Define the output directory for Parquet files
    output_dir = '/home/jovyan/work/marc/parquet'
    os.makedirs(output_dir, exist_ok=True)

    # Determine the output file name: replace .xml with .parquet
    base = os.path.basename(file)
    output_file = os.path.join(output_dir, base.replace('.xml', '.parquet'))

    if file.endswith('.xml'):
        # Convert XML to MARC (MRC) format first using parse_xml_to_array.
        try:
            # Open XML file using utf-8 encoding instead of binary mode.
            with open(file, 'r', encoding='utf-8') as f_in:
                xml_string = f_in.read()
            # Encode to bytes if the parser requires bytes input.
            records = parse_xml_to_array(xml_string.encode('utf-8'))
            logger.info(f"Parsed {len(records)} MARC records from XML")
        except Exception as e:
            logger.error(f"Error processing XML file {file} with utf-8 encoding: {e}")
            return False

        try:
            with tempfile.NamedTemporaryFile(delete=False) as temp:
                temp_file = temp.name
            with open(temp_file, 'wb') as f_temp:
                for record in records:
                    if not isinstance(record, Record):
                        raise ValueError("Invalid MARC record")
                    f_temp.write(record.as_marc())
            logger.info(f"Wrote temporary MRC file {temp_file}")
        except Exception as e:
            logger.error(f"Error converting XML to MRC for file {file}: {e}")
            if os.path.exists(temp_file):
                os.remove(temp_file)
            return False

        # Run marctable on the temporary MRC file.
        logger.info(f"Running marctable command: marctable parquet {temp_file} {output_file}")
        exit_status = os.system(f'marctable parquet {temp_file} {output_file}')
        if exit_status != 0:
            logger.error(f"Error executing marctable command for file {file}")
            os.remove(temp_file)
            return False
        else:
            logger.info(f"Created Parquet file {output_file}")
        os.remove(temp_file)
        return True

    else:
        # Process non-XML files (for example, raw MRC files).
        try:
            with tempfile.NamedTemporaryFile(delete=False) as temp:
                temp_file = temp.name
        except Exception as e:
            logger.error(f"Error creating temporary file for {file}: {e}")
            return False

        try:
            with open(file, 'rb') as f_in, open(temp_file, 'wb') as temp_out:
                from pymarc import MARCReader
                reader = MARCReader(f_in, to_unicode=True, force_utf8=True, utf8_handling='replace')
                for record in reader:
                    if not isinstance(record, Record):
                        raise ValueError("Invalid MARC record")
                    temp_out.write(record.as_marc())
        except Exception as e:
            logger.error(f"Error processing file {file}: {e}")
            os.remove(temp_file)
            return False

        logger.info(f"Running marctable command: marctable parquet {temp_file} {output_file}")
        exit_status = os.system(f'marctable parquet {temp_file} {output_file}')
        if exit_status != 0:
            logger.error(f"Error executing marctable command for file {file}")
            os.remove(temp_file)
            return False
        else:
            logger.info(f"Created Parquet file {output_file}")

        os.remove(temp_file)
        return True

def marc2parquet():
    files = get_files()
    results = []
    for file in files:
        # Determine output file name based on file extension.
        base = os.path.basename(file)
        output_file = os.path.join('/home/jovyan/work/marc/parquet', base.replace('.xml', '.parquet'))
        if os.path.exists(output_file):
            logger.info(f"Skipping already processed file {file}")
            continue
        result = process_file(file)
        results.append(result)
    successful_files = sum(results)
    logger.info(f"Processed {len(results)} files, {successful_files} successful, {len(results) - successful_files} failed")
    return all(results)

# Run the function and capture the result
result = marc2parquet()
logger.info(f"marc2parquet result: {result}")

# Load Parquet
The main benefits of Parquet (like predicate pushdown and column pruning) are realized during file I/O.

# Load Parquet to Spark DataFrame

In [3]:
spark_df_penn = spark.read.parquet('/home/jovyan/work/marc/parquet/penn-2022-07-20-full-marc21.parquet')
spark_df_brown = spark.read.parquet('/home/jovyan/work/marc/parquet/brown-2022-06-14-full-marc21.parquet')
spark_df_chicago = spark.read.parquet('/home/jovyan/work/marc/parquet/chicago-2022-06-22-full-marc21.parquet')
spark_df_columbia = spark.read.parquet('/home/jovyan/work/marc/parquet/columbia.parquet')
spark_df_cornell = spark.read.parquet('/home/jovyan/work/marc/parquet/cornell-2023-04-28-full-marc21.parquet')
spark_df_dartmouth = spark.read.parquet('/home/jovyan/work/marc/parquet/dartmouth-2022-08-19-full-marc21.parquet')
spark_df_duke = spark.read.parquet('/home/jovyan/work/marc/parquet/duke-2025-01-29-full-marc21.parquet_clean_rev.parquet')
spark_df_harvard = spark.read.parquet('/home/jovyan/work/marc/parquet/harvard.parquet')
spark_df_jhu = spark.read.parquet('/home/jovyan/work/marc/parquet/jhu-2023-08-23-full-marc21.parquet')
spark_df_mit = spark.read.parquet('/home/jovyan/work/marc/parquet/mit-marc21.parquet')
spark_df_princeton = spark.read.parquet('/home/jovyan/work/marc/parquet/princeton-2022-06-17-full-marc21.parquet')
spark_df_stanford = spark.read.parquet('/home/jovyan/work/marc/parquet/stanford-2024-08-28-full-marc21.parquet')
spark_df_yale = spark.read.parquet('/home/jovyan/work/marc/parquet/yale-2022-06-17-full-marc21.parquet')


# Import matching libraries

In [4]:
!pip install fuzzywuzzy
!pip install python-Levenshtein
%pip install langdetect

Note: you may need to restart the kernel to use updated packages.


# Find unique by Matchkey and lccn / isbn
Matchkey: 1. normalized title 2. edition and 3. publication fields. Matchkey is used when 010 (lccn) or 020 (isbn) fields are not available.

In [ ]:
import pyspark.sql.functions as F
from pyspark.sql.types import StringType, BooleanType, ArrayType
from pyspark.sql.functions import udf, col, collect_set, explode
import re
import unicodedata
from langdetect import detect

# 1. Create temp views
spark_df_penn.createOrReplaceTempView("penn")
spark_df_brown.createOrReplaceTempView("brown")
spark_df_chicago.createOrReplaceTempView("chicago")
spark_df_columbia.createOrReplaceTempView("columbia")
spark_df_cornell.createOrReplaceTempView("cornell")
spark_df_dartmouth.createOrReplaceTempView("dartmouth")
spark_df_duke.createOrReplaceTempView("duke")
spark_df_harvard.createOrReplaceTempView("harvard")
spark_df_jhu.createOrReplaceTempView("jhu")
spark_df_mit.createOrReplaceTempView("mit")
spark_df_princeton.createOrReplaceTempView("princeton")
spark_df_stanford.createOrReplaceTempView("stanford")
spark_df_yale.createOrReplaceTempView("yale")

# 2. Query to get fields from each view.
# Include F010 (LCCN), F020 (ISBN), F245, F250 (edition), and F260 (publication/distribution).
penn_245 = spark.sql("SELECT F001, F007, F010, F020, F245, F250, F260 FROM penn")
brown_245 = spark.sql("SELECT F001, F007, F010, F020, F245, F250, F260 FROM brown")
chicago_245 = spark.sql("SELECT F001, F007, F010, F020, F245, F250, F260 FROM chicago")
columbia_245 = spark.sql("SELECT F001, F010, F020, F245, F250, F260 FROM columbia")
cornell_245 = spark.sql("SELECT F001, F007, F010, F020, F245, F250, F260 FROM cornell")
dartmouth_245 = spark.sql("SELECT F001, F007, F010, F020, F245, F250, F260 FROM dartmouth")
duke_245 = spark.sql("SELECT F001, F007, F010, F020, F245, F250, F260 FROM duke")
harvard_245 = spark.sql("SELECT F001, F010, F020, F245, F250, F260 FROM harvard")
jhu_245 = spark.sql("SELECT F001, F007, F010, F020, F245, F250, F260 FROM jhu")
mit_245 = spark.sql("SELECT F001, F007, F010, F020, F245, F250, F260 FROM mit")
princeton_245 = spark.sql("SELECT F001, F007, F010, F020, F245, F250, F260 FROM princeton")
stanford_245 = spark.sql("SELECT F001, F007, F010, F020, F245, F250, F260 FROM stanford")
yale_245 = spark.sql("SELECT F001, F007, F010, F020, F245, F250, F260 FROM yale")

# 3. Normalize titles (F245), edition (F250), and publication/distribution (F260)
# Remove diacritics, punctuation, lowercase, and extra whitespace.
def normalize(text):
    if text:
        # If text is a list, join its elements into a string.
        if isinstance(text, list):
            text = " ".join(text)
        try:
            lang = detect(text)
        except Exception:
            lang = "unknown"
        if lang != "en":
            # For non-English, keep diacritics; only lowercase and trim whitespace.
            text = text.lower().strip()
            text = re.sub(r'\s+', ' ', text)
            return text
        else:
            # English: remove diacritics and punctuation.
            text = unicodedata.normalize('NFD', text)
            text = ''.join([c for c in text if not unicodedata.combining(c)])
            text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
            text = text.lower().strip()
            text = re.sub(r'\s+', ' ', text)
            return text
    return None

normalize_udf = udf(normalize, StringType())

def normalized_df(df):
    return df.withColumn("normalized_title", normalize_udf(F.col("F245"))) \
             .withColumn("normalized_edition", normalize_udf(F.col("F250"))) \
             .withColumn("normalized_pub", normalize_udf(F.col("F260")))

# Apply normalization to each DataFrame
penn_245 = normalized_df(penn_245)
brown_245 = normalized_df(brown_245)
chicago_245 = normalized_df(chicago_245)
columbia_245 = normalized_df(columbia_245)
cornell_245 = normalized_df(cornell_245)
dartmouth_245 = normalized_df(dartmouth_245)
duke_245 = normalized_df(duke_245)
harvard_245 = normalized_df(harvard_245)
jhu_245 = normalized_df(jhu_245)
mit_245 = normalized_df(mit_245)
princeton_245 = normalized_df(princeton_245)
stanford_245 = normalized_df(stanford_245)
yale_245 = normalized_df(yale_245)

# 4. Create a match key from the normalized title, edition, and publication/distribution.
def create_match_key(title, edition, pub):
    if title:
        key_parts = [title]
        if edition:
            key_parts.append(edition)
        if pub:
            key_parts.append(pub)
        return "_".join(key_parts)
    return None

match_key_udf = udf(create_match_key, StringType())

# 5. Add source column and a match key column
penn_df = penn_245.withColumn("source", F.lit("penn")) \
                  .withColumn("match_key", match_key_udf(F.col("normalized_title"), F.col("normalized_edition"), F.col("normalized_pub")))
brown_df = brown_245.withColumn("source", F.lit("brown")) \
                    .withColumn("match_key", match_key_udf(F.col("normalized_title"), F.col("normalized_edition"), F.col("normalized_pub")))
chicago_df = chicago_245.withColumn("source", F.lit("chicago")) \
                        .withColumn("match_key", match_key_udf(F.col("normalized_title"), F.col("normalized_edition"), F.col("normalized_pub")))
columbia_df = columbia_245.withColumn("source", F.lit("columbia")) \
                          .withColumn("match_key", match_key_udf(F.col("normalized_title"), F.col("normalized_edition"), F.col("normalized_pub")))
cornell_df = cornell_245.withColumn("source", F.lit("cornell")) \
                        .withColumn("match_key", match_key_udf(F.col("normalized_title"), F.col("normalized_edition"), F.col("normalized_pub")))
dartmouth_df = dartmouth_245.withColumn("source", F.lit("dartmouth")) \
                            .withColumn("match_key", match_key_udf(F.col("normalized_title"), F.col("normalized_edition"), F.col("normalized_pub")))
duke_df = duke_245.withColumn("source", F.lit("duke")) \
                  .withColumn("match_key", match_key_udf(F.col("normalized_title"), F.col("normalized_edition"), F.col("normalized_pub")))
harvard_df = harvard_245.withColumn("source", F.lit("harvard")) \
                        .withColumn("match_key", match_key_udf(F.col("normalized_title"), F.col("normalized_edition"), F.col("normalized_pub")))
jhu_df = jhu_245.withColumn("source", F.lit("jhu")) \
                .withColumn("match_key", match_key_udf(F.col("normalized_title"), F.col("normalized_edition"), F.col("normalized_pub")))
mit_df = mit_245.withColumn("source", F.lit("mit")) \
                .withColumn("match_key", match_key_udf(F.col("normalized_title"), F.col("normalized_edition"), F.col("normalized_pub")))
princeton_df = princeton_245.withColumn("source", F.lit("princeton")) \
                            .withColumn("match_key", match_key_udf(F.col("normalized_title"), F.col("normalized_edition"), F.col("normalized_pub")))
stanford_df = stanford_245.withColumn("source", F.lit("stanford")) \
                           .withColumn("match_key", match_key_udf(F.col("normalized_title"), F.col("normalized_edition"), F.col("normalized_pub")))
yale_df = yale_245.withColumn("source", F.lit("yale")) \
                   .withColumn("match_key", match_key_udf(F.col("normalized_title"), F.col("normalized_edition"), F.col("normalized_pub")))

# 6. Union all DataFrames
all_df = (penn_df.union(brown_df).union(chicago_df).union(cornell_df)
          .union(dartmouth_df).union(jhu_df).union(mit_df).union(princeton_df)
          .union(stanford_df).union(yale_df).union(duke_df))

# 7. Create a helper UDF to extract ISBN and LCCN values as a trimmed array.
def get_ids(f010, f020):
    ret = []
    # Process F010
    if f010:
        if isinstance(f010, list):
            for item in f010:
                if item and isinstance(item, str):
                    trimmed = item.strip()
                    if trimmed:
                        ret.append(trimmed)
        elif isinstance(f010, str):
            trimmed = f010.strip()
            if trimmed:
                ret.append(trimmed)
    # Process F020
    if f020:
        if isinstance(f020, list):
            for item in f020:
                if item and isinstance(item, str):
                    trimmed = item.strip()
                    if trimmed:
                        ret.append(trimmed)
        elif isinstance(f020, str):
            trimmed = f020.strip()
            if trimmed:
                ret.append(trimmed)
    return ret

get_ids_udf = udf(get_ids, ArrayType(StringType()))

# 8. Add an id_list column (with ISBN and/or LCCN) to all records.
all_df = all_df.withColumn("id_list", get_ids_udf(F.col("F010"), F.col("F020")))

# 9. Define the matching key array.
# If any ISBN or LCCN identifiers exist, use them; otherwise, use the match key.
all_df = all_df.withColumn("key_array",
    F.when(F.size(F.col("id_list")) > 0, F.col("id_list"))
     .otherwise(F.array(F.col("match_key")))
)

# 10. Explode the key_array so that each record has one key per identifier (or one key if using the match key).
all_df_exploded = all_df.withColumn("key", explode("key_array"))

# 11. Group by key and collect sources where that key appears.
grouped = all_df_exploded.groupBy("key").agg(
    collect_set("source").alias("sources"),
    F.count("*").alias("record_count")
)

# 12. Define the complete set of sources.
all_sources = {"penn", "brown", "chicago", "cornell", "dartmouth",
               "duke", "jhu", "mit", "princeton", "stanford", "yale"}

# 13. Identify common records: groups where the key appears in all sources.
def has_all_sources(sources):
    return all(src in sources for src in all_sources)

has_all_sources_udf = udf(has_all_sources, BooleanType())
common_fuzzy = grouped.filter(has_all_sources_udf(F.col("sources")))

# 14. Count common records.
common_fuzzy_count = common_fuzzy.count()
print(f"Number of fuzzy common (all sources) records: {common_fuzzy_count}")

# 15. Derive unique records:
# Perform an anti-join to remove records that are common across all sources,
# then deduplicate based on the match_key so that only one copy of the same book remains.
unique_exploded = all_df_exploded.join(common_fuzzy.select("key"), on="key", how="left_anti")
unique_df = unique_exploded.dropDuplicates(["match_key"])

# 16. Count unique records per source.
unique_penn = unique_df.filter(F.col("source") == "penn")
print(f"Number of unique fuzzy records in Penn: {unique_penn.count()}")
unique_brown = unique_df.filter(F.col("source") == "brown")
print(f"Number of unique fuzzy records in Brown: {unique_brown.count()}")
unique_chicago = unique_df.filter(F.col("source") == "chicago")
print(f"Number of unique fuzzy records in Chicago: {unique_chicago.count()}")
unique_columbia = unique_df.filter(F.col("source") == "columbia")
print(f"Number of unique fuzzy records in Columbia: {unique_columbia.count()}")
unique_cornell = unique_df.filter(F.col("source") == "cornell")
print(f"Number of unique fuzzy records in Cornell: {unique_cornell.count()}")
unique_dartmouth = unique_df.filter(F.col("source") == "dartmouth")
print(f"Number of unique fuzzy records in Dartmouth: {unique_dartmouth.count()}")
unique_duke = unique_df.filter(F.col("source") == "duke")
print(f"Number of unique fuzzy records in Duke: {unique_duke.count()}")
unique_harvard = unique_df.filter(F.col("source") == "harvard")
print(f"Number of unique fuzzy records in Harvard: {unique_harvard.count()}")
unique_jhu = unique_df.filter(F.col("source") == "jhu")
print(f"Number of unique fuzzy records in JHU: {unique_jhu.count()}")
unique_mit = unique_df.filter(F.col("source") == "mit")
print(f"Number of unique fuzzy records in MIT: {unique_mit.count()}")
unique_princeton = unique_df.filter(F.col("source") == "princeton")
print(f"Number of unique fuzzy records in Princeton: {unique_princeton.count()}")
unique_stanford = unique_df.filter(F.col("source") == "stanford")
print(f"Number of unique fuzzy records in Stanford: {unique_stanford.count()}")
unique_yale = unique_df.filter(F.col("source") == "yale")
print(f"Number of unique fuzzy records in Yale: {unique_yale.count()}")

# 17. Save results
unique_df.select("F001", "F245", "source").write.mode("overwrite") \
    .parquet("/home/jovyan/work/marc/unique_fuzzy_records.parquet")
common_fuzzy.select("key", "sources", "record_count").write.mode("overwrite") \
    .parquet("/home/jovyan/work/marc/common_fuzzy_records.parquet")
unique_penn.write.mode("overwrite").parquet("/home/jovyan/work/marc/unique_penn.parquet")
unique_brown.write.mode("overwrite").parquet("/home/jovyan/work/marc/unique_brown.parquet")
unique_chicago.write.mode("overwrite").parquet("/home/jovyan/work/marc/unique_chicago.parquet")
unique_columbia.write.mode("overwrite").parquet("/home/jovyan/work/marc/unique_columbia.parquet")
unique_cornell.write.mode("overwrite").parquet("/home/jovyan/work/marc/unique_cornell.parquet")
unique_dartmouth.write.mode("overwrite").parquet("/home/jovyan/work/marc/unique_dartmouth.parquet")
unique_duke.write.mode("overwrite").parquet("/home/jovyan/work/marc/unique_duke.parquet")
unique_harvard.write.mode("overwrite").parquet("/home/jovyan/work/marc/unique_harvard.parquet")
unique_jhu.write.mode("overwrite").parquet("/home/jovyan/work/marc/unique_jhu.parquet")
unique_mit.write.mode("overwrite").parquet("/home/jovyan/work/marc/unique_mit.parquet")
unique_princeton.write.mode("overwrite").parquet("/home/jovyan/work/marc/unique_princeton.parquet")
unique_stanford.write.mode("overwrite").parquet("/home/jovyan/work/marc/unique_stanford.parquet")
unique_yale.write.mode("overwrite").parquet("/home/jovyan/work/marc/unique_yale.parquet")


# Analyze results

In [ ]:
# analyze the unique records from penn
unique_penn = spark.read.parquet('/home/jovyan/work/marc/unique_penn.parquet')

# Create temp view
unique_penn.createOrReplaceTempView("unique_penn")

# we are going to add a column to support sorting by 007 format


In [ ]:
# how many unique records are there in the Penn rarest titles?
print(f"Number of unique records in Penn rarest titles: {rarest_titles.count()}")


In [ ]:
# Exclude electronic resources
unique_print_penn = unique_penn.filter(F.substring(F.col("F007").getItem(0), 1, 1) != "c")

# Further include only records where F007/00 equals "t" (identifies the item physically as a text)
unique_text_print_penn = unique_print_penn.filter(F.substring(F.col("F007").getItem(0), 1, 1) == "t")

# Now select the rarest titles (ordering by F245 here is only illustrative; adjust as needed)
rarest_titles = unique_text_print_penn.select("F001", "F245").orderBy(F.col("F245").desc())
rarest_titles.show(10)

In [ ]:
# Exclude electronic resources
unique_print_penn = unique_penn.filter(F.substring(F.col("F007").getItem(0), 1, 1) != "c")

# Further include only records where F007/00 equals "t" (identifies the item physically as a text)
unique_text_print_penn = unique_print_penn.filter(F.substring(F.col("F007").getItem(0), 1, 1) == "t")

# Now select the rarest titles (ordering by F245 here is only illustrative; adjust as needed)
rarest_titles = unique_text_print_penn.select("F001", "F245").orderBy(F.col("F245").desc())
rarest_titles.show(10)

In [ ]:
# count the number of unique records in the Penn rarest text titles
print(f"Number of unique records in Penn rarest titles: {rarest_titles.count()}")

In [ ]:
# export excel file 
rarest_titles.toPandas().to_excel("/home/jovyan/work/marc/penn_rarest_titles_text.xlsx", index=False)